In [1]:
#25th July 2025 
#API Test Script 

import requests

# Keys and IDs
# NOTE: Contains API keys and passwords 
tmdb_api_key = "ef74866feee9084817794614ffbae21d"
opensub_api_key = "V8cxCT3494611SyHgQRVHbetkQEZVTHW"
opensub_username = "rishabh_bijani"
opensub_password = "Rishabhb2004!"  
sample_imdb_id = "tt8207768"

#API Test Key 
print("\n🔍 Testing TMDb API...")
tmdb_url = f"https://api.themoviedb.org/3/find/{sample_imdb_id}?api_key={tmdb_api_key}&external_source=imdb_id"
try:
    resp = requests.get(tmdb_url)
    print("TMDb Status Code:", resp.status_code)
    if resp.status_code == 200:
        overview = resp.json()["movie_results"][0].get("overview", "")
        print("✅ TMDb description retrieved.")
        print("Overview:", overview[:200], "...")
    elif resp.status_code == 401:
        print("❌ TMDb: Invalid API key.")
    elif resp.status_code == 403:
        print("❌ TMDb: Blocked (VPN may be required).")
    else:
        print("⚠️ TMDb Error:", resp.text)
except Exception as e:
    print(f"❌ TMDb Request failed: {e}")

#OpenSubtitles API Test
print("\n🔐 Testing OpenSubtitles login with token...")
login_url = "https://api.opensubtitles.com/api/v1/login"
headers_login = {
    "Api-Key": opensub_api_key,
    "Content-Type": "application/json",
    "User-Agent": "RAApp/1.0.0"
}
login_payload = {
    "username": opensub_username,
    "password": opensub_password
}

try:
    login_resp = requests.post(login_url, headers=headers_login, json=login_payload)
    print("Login Status Code:", login_resp.status_code)
    if login_resp.status_code == 200:
        token = login_resp.json()["token"]
        print("✅ OpenSubtitles token acquired.")
    else:
        print("❌ Login failed:", login_resp.text)
        token = None
except Exception as e:
    print(f"❌ OpenSubtitles Login Error: {e}")
    token = None

#OpenSubtitles Search and Download Test
if token:
    print("\n🔎 Searching subtitles for", sample_imdb_id)
    headers_auth = {
        "Authorization": f"Bearer {token}",
        "Api-Key": opensub_api_key,
        "Content-Type": "application/json",
        "Accept": "application/json",
        "User-Agent": "RAApp/1.0.0"
    }
    search_url = f"https://api.opensubtitles.com/api/v1/subtitles?imdb_id={sample_imdb_id[2:]}&languages=en"

    try:
        sub_resp = requests.get(search_url, headers=headers_auth)
        print("Subtitle Search Status:", sub_resp.status_code)
        if sub_resp.status_code == 200:
            results = sub_resp.json().get("data", [])
            print(f"✅ Found {len(results)} subtitles.")
            if results:
                file_id = results[0]["attributes"]["files"][0]["file_id"]
                print("Top subtitle file ID:", file_id)

                # Download .srt link
                download_url = "https://api.opensubtitles.com/api/v1/download"
                dl_resp = requests.post(download_url, headers=headers_auth, json={"file_id": file_id})
                if dl_resp.status_code == 200:
                    dl_link = dl_resp.json()["link"]
                    print("✅ Subtitle download link:", dl_link)

                    # Optional actual download
                    srt = requests.get(dl_link)
                    if srt.status_code == 200:
                        with open(f"{sample_imdb_id}.srt", "wb") as f:
                            f.write(srt.content)
                        print(f"✅ Subtitle file saved as {sample_imdb_id}.srt")
                    else:
                        print("❌ Failed to retrieve subtitle file.")
                else:
                    print("❌ Failed to generate download link.")
        else:
            print("❌ Subtitle search failed:", sub_resp.text)
    except Exception as e:
        print(f"❌ Subtitle search error: {e}")


🔍 Testing TMDb API...
TMDb Status Code: 200
✅ TMDb description retrieved.
Overview: A story about four children living in a Mumbai slum in India. An eight-year old Kanhu writes a letter to the Prime Minister after a dramatic incident with his mother. ...

🔐 Testing OpenSubtitles login with token...
Login Status Code: 200
✅ OpenSubtitles token acquired.

🔎 Searching subtitles for tt8207768
Subtitle Search Status: 200
✅ Found 2 subtitles.
Top subtitle file ID: 8488379
✅ Subtitle download link: https://www.opensubtitles.com/download/D2AEBFA1683FC319CDE02DBD9822CEC3CA5505024CE654E917DCEB91D05CE41E15196A20A0D3E40FCC1457E30F61491BAB91BC3BCEB978B9C36E31BE1E155E59E07519312BC423543FE74F2ACC22473089C5E88DE10C65DB398677DD492E3B8E21703B19BE1578CE72FDF62C882DA3BAD1725EB5A8CD42A4BE4B71BFCF2519064F3EBC0D20D4FE21510D28974EDA36ABB210AD773FFB0D7A6A38B1CEBA3C248DDAAE31D8C19B5CEF11C5592419BB5B8ADBFA089F8EEAD5F9A9F131132BE48F24BEDA7A51F761DC1ED37FAC7E07E615E8D16B941A5AD97F185B9776F4AC5B567EA97A336D36E994C